# Premium Allocation Tool

In [ ]:
import numpy as np
import pandas as pd
import json
import uuid  

from pat_back import PatJob, PatFlag, AppSettings, PatHelper

# Get job list 


## Delete Job

In [ ]:
# pd.DataFrame(json.loads(PatJob.get_job_list()))
lst= [a for a in range(100000233,100000300)]
PatHelper.delete(*lst)

## Job parameters

In [ ]:
ret = PatJob.get_job_para(100000197)
if ret:
    print(json.dumps(json.loads(ret),indent=4))


## Result summary 

In [ ]:
df = PatJob.get_summary(100000197)
df

## Results

In [ ]:
df = PatJob.get_results(100000197)
df

## Validation data

In [ ]:
df1,df2,df3 = PatJob.get_validation_data(100000197)
print(len(df1), len(df2), len(df3))

In [ ]:
df2.sort_values(by='LocationIDStack')

# Create New Job 

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid']=str(uuid.uuid4())

job = PatJob(job_para)
job.job_id

# Process

In [ ]:
job.perform_analysis()

# Testing

In [ ]:

import json
import numpy as np
import pandas as pd
import pyodbc
from bcpandas import SqlCreds, to_sql

df = pd.read_csv('data/psold_aoi.csv')

creds = SqlCreds(AppSettings.PAT_JOB_SVR, AppSettings.PAT_JOB_DB, AppSettings.PAT_JOB_USR, AppSettings.PAT_JOB_PWD)
to_sql(df,'psold_aoi', creds, index=False, if_exists='append') 

In [ ]:
job_conn = f'''DRIVER={{SQL Server}};Server={AppSettings.PAT_JOB_SVR};Database={AppSettings.PAT_JOB_DB};
            User Id={AppSettings.PAT_JOB_USR};Password={AppSettings.PAT_JOB_PWD};
            MultipleActiveResultSets=true;'''
with pyodbc.connect(job_conn) as conn:
             dfHPR = pd.read_sql_query(f"""select * from psold_hpr order by limit""", conn)
dfHPR

# Start

In [2]:
from datetime import datetime
import numpy as np
import pandas as pd

from pat_back import Psold


In [21]:
aoi_split = np.loadtxt(r"C:\_Working\PAT_20201019\debug\AOI_R.csv")
df_psold = pd.read_csv("C:\_Working\PAT_20201019\debug\ISOt_2016.csv")
df_psold = df_psold[(df_psold.COVG == 4) & (df_psold.SUBGRP == 4)][
    list(('RG','EG','AOCC',*[f'G{i}' for i in range(1, 12)]))]
df_psold.columns=['RG', 'EG', 'OCC', *[f'W{i}' for i in range(1, 12)]]

psold  = Psold(df_psold, aoi_split, 1.035, datetime(2015,12,31))


df_wts = pd.read_csv(r"C:\_Working\PAT_20201019\debug\psold_weight.csv").set_index('RG')
df_wts.loc[1:39,['PremiumWeight']] = 0
df_wts.loc[39,['PremiumWeight']] = 1

DT = pd.read_csv("C:\_Working\PAT_20201019\debug\dt_2020.csv").iloc[:,:10].rename(columns={
            'polID': 'PolicyID',
            'rawTIV' : 'TIV',
            'limit' : 'Limit',
            'prem' : 'PolPrem', 
            'SIRD':'Retention',
            'LR': 'LossRatio',
            'stack': 'LocStack',
            'rtG': 'RatingGroup'
        })
DT.reset_index(inplace= True, drop=True)
DT.reset_index(inplace=True)

# # # df_hpr = pd.read_csv("C:\_Working\PAT_20201019\debug\HPRBlend.csv") if hpr else None



# # # DT = DT.sort_values('polDed')#.reset_index(drop=True)
# df = psold.pat(DT, df_wts, avg_acc_date=datetime(2023,1,1), addt_cvg=0.5)
# df.set_index('index').sort_index()[['locID','AllocPrem']]
DT

,index,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,LocStack,RatingGroup,locID
0,0,1,1000000.0,0.0,500000.0,0.8,0.2,8000000.0,990.0,1.0,1
1,1,2,1000000.0,1000000.0,800000.0,0.7,0.4,4000000.0,990.0,2.0,2
2,2,3,3000000.0,2000000.0,1000000.0,0.6,0.6,12000000.0,990.0,3.0,3
3,3,4,5000000.0,5000000.0,1500000.0,0.7,0.8,295000.0,990.0,4.0,4
4,4,5,500000.0,1000000.0,250000.0,0.8,1.0,1110000.0,990.0,5.0,5
5,5,6,10000000.0,10000000.0,3000000.0,0.4,1.0,100000000.0,990.0,NaN,6
6,6,7,25000000.0,0.0,8000000.0,0.2,1.0,250000000.0,990.0,NaN,7
7,7,8,5000000.0,20000000.0,1000000.0,0.9,1.0,22000000.0,990.0,NaN,8
8,8,9,250000.0,2000000.0,25000.0,0.1,1.0,800000.0,990.0,NaN,9
9,9,10,250000.0,2250000.0,200000.0,1.1,1.0,1000000.0,991.0,NaN,10


In [8]:
DT


,index,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,LocStack,RatingGroup,locID
0,0,1,1000000.0,0.0,500000.0,0.8,0.2,8000000.0,990.0,1.0,1
1,1,2,1000000.0,1000000.0,800000.0,0.7,0.4,4000000.0,990.0,2.0,2
2,2,3,3000000.0,2000000.0,1000000.0,0.6,0.6,12000000.0,990.0,3.0,3
3,3,4,5000000.0,5000000.0,1500000.0,0.7,0.8,295000.0,990.0,4.0,4
4,4,5,500000.0,1000000.0,250000.0,0.8,1.0,1110000.0,990.0,5.0,5
5,5,6,10000000.0,10000000.0,3000000.0,0.4,1.0,100000000.0,990.0,NaN,6
6,6,7,25000000.0,0.0,8000000.0,0.2,1.0,250000000.0,990.0,NaN,7
7,7,8,5000000.0,20000000.0,1000000.0,0.9,1.0,22000000.0,990.0,NaN,8
8,8,9,250000.0,2000000.0,25000.0,0.1,1.0,800000.0,990.0,NaN,9
9,9,10,250000.0,2250000.0,200000.0,1.1,1.0,1000000.0,991.0,NaN,10


In [3]:
Limit 	 Retention 	 Allocated Premium 	 Participation 	 Building 	 Contents 	 BI 	 AOI 	 Rating Group 	 ACCGRPID 	 Original Location ID 	 Original Policy ID 	 PseudoPolicyID 	 Pseudo Layer ID 	 PolicyLimitLAS 	 PolicyAttachLAS 	 Diff 
0,1,412683176,5.48, 1000, 1, 1,1000, 990, 35, 1
1,2,412683181.5, 5.48	241.4142304	1	0	0	1000	1000	35	91621	4390520	143785	143785_4390520	1	2863.12349	5.479509774	


#  Limit 	 Retention 	 Allocated Premium 	 Participation 	 Building 	 Contents 	 BI 	 AOI 	 Rating Group 	 ACCGRPID 	 Original Location ID 	 Original Policy ID 	 PseudoPolicyID 	 Pseudo Layer ID 	 PolicyLimitLAS 	 PolicyAttachLAS 	 Diff 
# 412683176	5.48	147.8823888	1	0	0	1000	1000	35	91621	4390520	143785	143785_4390520	1	1693.976937	5.47267952	1169.146553
# 412683181.5	5.48	241.4142304	1	0	0	1000	1000	35	91621	4390520	143785	143785_4390520	1	2863.12349	5.479509774	


,Limit,Weight
0,0.000000e+00,0.00
1,2.000000e+07,0.00
2,3.000000e+07,0.25
3,5.000000e+07,0.50
4,1.000000e+08,1.00
5,1.000000e+33,1.00


In [22]:
DT = pd.DataFrame(data=[[0,1,412683176,5.48, 1000, 1, 1,999.9, 990, 35, 1]], columns=DT.columns)
DT

,index,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,LocStack,RatingGroup,locID
0,0,1,412683176,5.48,1000,1,1,999.9,990,35,1


In [23]:
df = psold.pat(DT, None, avg_acc_date=datetime(2022,1,1), addt_cvg=2)
df

,index,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,LocStack,RatingGroup,locID,Lx,Policy,EffLmt,AOIr,PolLAS,DedLAS,GuLimLAS,GuDedLAS,AllocPrem
0,0,1,412683176,5.48,1000,1,1,999.9,990,35,1,4.126832e+08,4.126832e+08,2999.7,0,2862.849917,5.47951,2862.849917,5.47951,1000.0


,index,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,LocStack,RatingGroup,locID,Lx,Policy,EffLmt,AOIr,PolLAS,DedLAS,GuLimLAS,GuDedLAS,AllocPrem
0,0,1,412683176,5.48,1000,1,1,1000,990,35,1,4.126832e+08,4.126832e+08,3000.0,0,2863.12349,5.47951,2863.12349,5.47951,1000.0


In [47]:
np.searchsorted(psold.aoi_split[1:-1], 10, side='left')

0

In [37]:
psold.aoi_split[-1] - 1e10

0.0